In [1]:
from epyc import JSONLabNotebook
import epyc
import epydemic
import numpy as np
from epydemic import SEIR, SIR, ERNetwork, Monitor, ProcessSequence
import pandas as pd
import matplotlib.pyplot as plt
from parameters import *
from scipy.stats import ks_2samp
from matplotlib.gridspec import SubplotSpec
import string
plt.rcParams.update({'font.size': 13})

In [2]:
json_lab = JSONLabNotebook(get_out_path('ex_1_sirq'), create=False)

In [3]:
dfs = [json_lab.dataframe(tag=rs) for rs in json_lab.resultSets()]
df = pd.concat(dfs)

In [5]:
kss = {2: [], 3: [], 4:[]}
ps = {2: [], 3: [], 4:[]}

for i, pD in enumerate(p_detect):
    expected_data_raw = np.load('limerick-data/' + str(n_RC) + "_" + str(pD) +'_unprocessed_fig3A.npy')

    for j, c in enumerate(c_sizes_RC) :
        ob = df.loc[(df["cSize"] == c) & (df["pDetect"] == pD)]
        ex = list(zip(*np.array(expected_data_raw[j])))
     
        mean_od = ob.groupby([SIR.P_INFECT], as_index=False)[[SIR.REMOVED, "epydemic.sir.Q"]].mean()
        mean_od_vals = (mean_od[SIR.REMOVED] + (mean_od["epydemic.sir.Q"] * mean_od[SIR.P_INFECT])) / n_RC
        mean_ed = np.array(ex).mean(axis=1)

        ks_statistic, p_value = ks_2samp(mean_ed, mean_od_vals)
        kss[c].append(round(ks_statistic,3))
        ps[c].append(round(p_value,3))
        
print("KS", kss)
print("P-Value", ps)

KS {2: [0.075, 0.125, 0.075], 3: [0.075, 0.075, 0.05], 4: [0.075, 0.05, 0.05]}
P-Value {2: [1.0, 0.919, 1.0], 3: [1.0, 1.0, 1.0], 4: [1.0, 1.0, 1.0]}
